# Negative Inversion

### How to use
1. Create a folder `clip_jpg/` containing the negative images. These are used for alignment because the original negative image holds more information.
2. In `clip_jpg/`, duplicate one image and rename it `ref.jpg`. This will be the reference image that all other images are aligned to.
3. Create a folder `clip_inverted` containing the inverted (positive) image.
4. Run the jupyter notebook. It will create a folder `clip_out/` containing the aligned images.

In [3]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import os

plt.style.use('dark_background')

In [5]:
# Change this (0-255) to set sprocket threshold value
sprocket_threshold = 200 

def findSprocketBox(img):
    t_img = img.copy()
    
    # Blur
    kernel_size = 10
    kernel = np.ones((kernel_size,kernel_size),np.float32)/(kernel_size*kernel_size)
    t_img = cv.filter2D(t_img,-1,kernel)
    
    # Threshold to get sprockets
    t_img = cv.cvtColor(t_img, cv.COLOR_BGR2GRAY)
    _, t_img = cv.threshold(t_img,sprocket_threshold,255,cv.THRESH_BINARY)

    # Apply mask to isolate sprockets
    (height, width) = t_img.shape
    mask = np.zeros(t_img.shape, np.uint8)
    mask[0:6*height//7,width//2:15*width//16] = 255
    t_img = cv.bitwise_and(t_img, t_img, mask=mask)

    # Find contours and get largest two contours (ie. the two sprockets)
    contours, _ = cv.findContours(t_img, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    ordered_contours = sorted(contours, key=cv.contourArea, reverse=True)
    cnt = np.concatenate(ordered_contours[:2])
    rect = cv.minAreaRect(cnt)
    box = cv.boxPoints(rect)
    box = np.int0(box)

    # Uncomment below for debugging
    new_img = cv.drawContours(img, [box], -1, (0,255,0), 10)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 18))
    ax1.imshow(new_img)
    ax2.imshow(t_img)

    # Make first point in box the bottom left point
    while (np.sum(box[0]) != np.min(np.sum(box, axis=1))):
        box = np.roll(box, -1, axis=0)
    
    # Return line segment of sprocket edge
    return box[:2]

In [ ]:
folder_name = "clip"

os.makedirs(folder_name+"_out", exist_ok=True)

ref_img = cv.imread(folder_name+"_jpg/ref.jpg", cv.COLOR_BGR2RGB)
ref_img = cv.cvtColor(ref_img, cv.COLOR_BGR2RGB)

ref_box = findSprocketBox(ref_img)

for index, file in enumerate(sorted(os.listdir(folder_name+"_inverted"))):
    try:
        print(file)

        img = cv.imread(folder_name+"_jpg/"+file, cv.COLOR_BGR2RGB)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        inv_img = cv.imread(folder_name+"_inverted/"+file, cv.COLOR_BGR2RGB)
        inv_img = cv.cvtColor(inv_img, cv.COLOR_BGR2RGB)

        box = findSprocketBox(img)

        # Transform to match reference image
        a,b = box
        c,d = ref_box

        lenAB = np.sqrt((b-a)[0]**2 + (b-a)[1]**2)
        lenCD = np.sqrt((d-c)[0]**2 + (d-c)[1]**2)

        trans_mat = np.array([
            [1,0,-a[0]],
            [0,1,-a[1]],
            [0,0,1]
        ])
        theta =  -np.arctan2(
            (d[0]-c[0])*(b[1]-a[1])-(d[1]-c[1])*(b[0]-a[0]), 
            (d[0]-c[0])*(b[0]-a[0])+(d[1]-c[1])*(b[1]-a[1])
        )
        rot_mat = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0,0,1]
        ])
        scale = lenCD/lenAB
        scale_mat = np.array([
            [scale, 0, 0],
            [0, scale, 0],
            [0, 0, 1]
        ])              
        trans_mat2 = np.array([
            [1,0,c[0]],
            [0,1,c[1]],
            [0,0,1]
        ])

        matrix = trans_mat2.dot(scale_mat).dot(rot_mat).dot(trans_mat)
        
        inv_img = cv.warpPerspective(inv_img, matrix, ref_img.shape[:2][::-1])

        cv.imwrite(folder_name+"_out/"+str(index).zfill(3)+".JPG", cv.cvtColor(inv_img, cv.COLOR_RGB2BGR))
        
    except:
        print("error")